In [46]:
import pandas as pd
import numpy as np

In [67]:
data = pd.read_csv("../data/AB_NYC_2019.csv")
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [1]:
data.info()
data.describe()
data.isna().sum()

NameError: name 'data' is not defined

In [69]:
data = data.drop(columns=["id", "latitude", "longitude"])

In [70]:
data["name"] = data["name"].fillna("Unknown")
data["last_review"] = data["last_review"].fillna("None")
data["reviews_per_month"] = data["reviews_per_month"].fillna(0)
data

,name,host_id,host_name,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,Private room,149,1,9,2018-10-19,0.21,6,365
1,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,Private room,150,3,0,None,0.00,1,365
3,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,Private room,70,2,0,None,0.00,2,9
48891,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,Private room,40,4,0,None,0.00,2,36
48892,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,Entire home/apt,115,10,0,None,0.00,1,27
48893,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,Shared room,55,1,0,None,0.00,6,2


In [ ]:
🟩 STEP 4 — Outlier Cleaning (Quantile Method)

In [74]:
data["price"].describe()

# Z-SCORE METHOD
mean = data["price"].mean()
std = data["price"].std()
z_scores = (data["price"] - mean) / std
data_z = data[np.abs(z_scores) < 3]
len(data) - len(data_z)

# IQR METHOD (Interquartile Range)
Q1 = data["price"].quantile(0.25)
Q3 = data["price"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

data_iqr = data[(data["price"] >= lower) & (data["price"] <= upper)]
len(data) - len(data_iqr)

# QUANTILE FILTERING (Percentile-Based)
low = data["price"].quantile(0.01)
high = data["price"].quantile(0.99)

data_q = data[(data["price"] >= low) & (data["price"] <= high)]
len(data) - len(data_q)


print("Original:", data.shape)
print("Z-score:", data_z.shape)
print("IQR:", data_iqr.shape)
print("Quantile:", data_q.shape)

data = data_q

Original: (48017, 13)
Z-score: (46895, 13)
IQR: (45505, 13)
Quantile: (47080, 13)


In [77]:
data = data[data["minimum_nights"] <= 365] 

In [79]:
data["room_type"] = data["room_type"].str.lower().str.strip()
data["neighbourhood_group"] = data["neighbourhood_group"].str.title()
data["neighbourhood"] = data["neighbourhood"].str.title()
data

,name,host_id,host_name,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,private room,149,1,9,2018-10-19,0.21,6,365
1,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,private room,150,3,0,None,0.00,1,365
3,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,private room,70,2,0,None,0.00,2,9
48891,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,private room,40,4,0,None,0.00,2,36
48892,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,entire home/apt,115,10,0,None,0.00,1,27
48893,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell'S Kitchen,shared room,55,1,0,None,0.00,6,2


In [80]:
data["availability_rate"] = data["availability_365"] / 365
data

,name,host_id,host_name,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,availability_rate
0,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,private room,149,1,9,2018-10-19,0.21,6,365,1.000000
1,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,entire home/apt,225,1,45,2019-05-21,0.38,2,355,0.972603
2,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,private room,150,3,0,None,0.00,1,365,1.000000
3,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,entire home/apt,89,1,270,2019-07-05,4.64,1,194,0.531507
4,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,entire home/apt,80,10,9,2018-11-19,0.10,1,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,private room,70,2,0,None,0.00,2,9,0.024658
48891,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,private room,40,4,0,None,0.00,2,36,0.098630
48892,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,entire home/apt,115,10,0,None,0.00,1,27,0.073973
48893,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell'S Kitchen,shared room,55,1,0,None,0.00,6,2,0.005479


In [82]:
data.to_csv("../data/airbnb_clean.csv", index=False)